In [1]:
import getpass
import os
import json
from typing import List, Literal, Optional
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.messages import get_buffer_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
from langchain_cohere import CohereEmbeddings

cohere_api_key = "CokXtTG3UuojM8LPnH0320yaj9bFI9Aypsj6I7fU"


    
recall_vector_store = InMemoryVectorStore(( CohereEmbeddings(
        cohere_api_key=cohere_api_key, model="embed-english-v3.0"
    )))


import uuid
def get_user_id(config: RunnableConfig) -> str:
    user_id = config["configurable"].get("user_id")
    if user_id is None:
        raise ValueError("User ID needs to be provided to save a memory.")

    return user_id

@tool
def save_recall_memory(memory: str, config: RunnableConfig) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    user_id = get_user_id(config)
    document = Document(
        page_content=memory, id=str(uuid.uuid4()), metadata={"user_id": user_id}
    )
    recall_vector_store.add_documents([document])
    return memory

@tool
def search_recall_memories(query: str, config: RunnableConfig) -> List[str]:
    """Search for relevant memories."""
    user_id = get_user_id(config)

    def _filter_function(doc: Document) -> bool:
        return doc.metadata.get("user_id") == user_id

    documents = recall_vector_store.similarity_search(
        query, k=3, filter=_filter_function
    )
    return [document.page_content for document in documents]

search = TavilySearchResults(max_results=1)
tools = [save_recall_memory, search_recall_memories, search]

class State(MessagesState):
    # add memories that will be retrieved based on the conversation context
    recall_memories: List[str]
    

ModuleNotFoundError: No module named 'langchain_community'